# Note book imports

In [1]:
from os.path import exists

# Constants

In [3]:
EXAMPLE_FILE = "../spamData/01_Processing/practice_email.txt"

In [13]:
with open(EXAMPLE_FILE, "r", encoding="latin-1") as ex_in:
    is_body = False
    lines = []

    for line in ex_in.readlines():
        if is_body:
            lines.append(line)
        elif line == "\n":
            is_body = True
email_body = "".join(lines)
print(email_body)


Dear Mr Still

Good tidings to you and all your staff for the festive season ahead (Christmas).
Now to the crux of the matter-in-hand: I am a fully qualified Santa Claus and am wondering whether you might consider me to run my own "Santa's Grotto" in your store.
But WAIT! You're probably thinking: "What makes him so special?"
Well, first of all, I have made several changes to the characterisation of Father Christmas. Rather than greeting the children with shouts of "Ho, ho, ho!" I prefer to whisper the phrase "Dependence is not unfathomable in this cruel world we live in". In addition, my gifts are ALL hand-made, ranging from felt hoops to vanilla-pod holders.
You will note also, from the enclosed sketch, that I have radically redesigned Santa's outfit and have renamed my character "Lord Buckles". Would you be interested in employing me? I promise NEVER to let you down.
I look forward to hearing from you.

Best wishes
Robin Cooper
[Excerpt from the book: The Timewaster Letters by Robi